# Modelling

## Importando bibliotecas

In [40]:
import pathlib
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

## Importando os dados do notebook limpo

In [41]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

/Users/luccahiratsuca/Github/Insper/4 Semestre/Machine Learning/ames-project/data


In [42]:
clean_data_path = DATA_DIR / 'processed' / 'feature_engineering.pkl'

In [43]:
df = pd.read_pickle(clean_data_path)
df.head()

,MS.SubClass_160,MS.SubClass_190,MS.SubClass_20,MS.SubClass_30,MS.SubClass_50,MS.SubClass_60,MS.SubClass_70,MS.SubClass_80,MS.SubClass_85,MS.SubClass_90,...,SalePrice,HasShed,HasAlley,Garage.Age,Remod.Age,House.Age,SqFtPerRoom,Total_Home_Quality,Total_Bathrooms,HighQualSF
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.332438,False,False,9.32135,50.0,9.221993,2.047004,9.0,1.436018,406.154465
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.021189,False,False,9.229883,49.0,9.132084,2.045485,9.0,1.0,485.497247
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.235528,False,False,9.501053,52.0,9.398614,1.865857,10.0,1.17555,319.234445
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.38739,False,False,8.555959,42.0,8.469389,1.480214,10.0,2.611568,483.847904
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,5.278525,False,False,4.613697,12.0,4.582673,1.504453,8.0,2.17555,370.63674


## Separando os dados de treino e teste

In [44]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
from sklearn.linear_model import LinearRegression

In [47]:
model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [48]:
ypred = model.predict(X_test)

In [49]:
from sklearn.metrics import mean_squared_error

RMSE = np.sqrt(mean_squared_error(y_test, ypred))

In [50]:
RMSE

0.05224633745459965

In [51]:
error_percent = 100 * (10**RMSE - 1)
print(f'Average error is {error_percent:.2f}%')

Average error is 12.78%
